In [140]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Toeknising the titles 
We will now decapatilise and tokenise(segment) the titles so that we can process them

In [141]:
#importing df of story rows rather that comments 
df = pd.read_csv('output.csv')


In [142]:
df['title'].head

<bound method NDFrame.head of 0          "What May Happen in the Next Hundred Years", f...
1               Getting Started with JavaScript Unit Testing
2          Armstrong, the Django-based and open-source ne...
3                       Why Web Reviewers Make Up Bad Things
4          You Weren't Meant to Have a Boss: The Cliff Notes
                                 ...                        
5351743    Scrapling: Fast, Adaptive Web Scraping for Python
5351744                         Monkeys Predict US Elections
5351745                                                  NaN
5351746                                                  NaN
5351747                                                  NaN
Name: title, Length: 5351748, dtype: object>

In [143]:
df = df.dropna(subset=['title'])

In [144]:
df = df.head(2500)

In [145]:
import pandas as pd
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent language detection results
DetectorFactory.seed = 0

# Function to check if the title is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False  # If detection fails, we assume it's not English

# Filter rows where the title is in English
df['title'] = df['title'].str.lower()
df = df[df['title'].apply(is_english)]

# Display the modified DataFrame
print(df)

           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [146]:
import pandas as pd
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download NLTK resources if you haven't done so already
nltk.download('punkt')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


# Function to tokenize and apply stemming/lemmatization
#def tokenize_and_process(text):
    #if not isinstance(text, str) or not text:  # Check if the text is a valid non-empty string
        #return ''  # Return empty string for non-string or empty values

    # Tokenize the text by whitespace
    #tokens = text.split()
    
    # Apply stemming and lemmatization
    #processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    
    #return ' '.join(processed_tokens)  # Return the processed tokens as a single string

# Function to tokenize without stemming/lemmatization
def tokenize_and_process(text):
    if not isinstance(text, str) or not text:  # Check if the text is a valid non-empty string
        return ''  # Return empty string for non-string or empty values

    # Tokenize the text by whitespace
    tokens = text.split()
    
    return ' '.join(tokens)  # Return the tokens as a single string


# Apply the function to create the 'adjusted_title' column
df['title'] = df['title'].apply(tokenize_and_process)

# Display the modified DataFrame
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [147]:
import pandas as pd
import re

# Function to replace punctuation with special tokens
def preprocess_text(text):
    # Replace specific punctuation with special tokens
    text = text.replace(',', '<COMMA>')  # Replace commas
    text = text.replace('.', '<FULLSTOP>')  # Replace full stops
    text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [148]:
import pandas as pd
import re


# Function to replace all punctuation with special tokens
def preprocess_text(text):
    # Replace various punctuation with special tokens
    replacements = {
        ',': '<COMMA>',
        '.': '<FULLSTOP>',
        '!': '<EXCLAMATION>',
        '?': '<QUESTION>',
        ';': '<SEMICOLON>',
        ':': '<COLON>',
        '(': '<OPENPAREN>',
        ')': '<CLOSEPAREN>',
        '[': '<OPENBRACKET>',
        ']': '<CLOSEBRACKET>',
        '{': '<OPENBRACE>',
        '}': '<CLOSEBRACE>',
        '\'': '<QUOTE>',
        '\"': '<DOUBLEQUOTE>',
        '-': '<DASH>',
        '_': '<UNDERLINE>',
        '…': '<ELLIPSIS>'  # Ellipsis
    }
    
    for punctuation, token in replacements.items():
        text = text.replace(punctuation, token)  # Replace each punctuation
    return text

# Tokenize the text (split into individual words and special tokens)
def tokenize_text(text):
    # Tokenize by splitting on whitespace and keeping punctuation tokens
    return re.findall(r'\S+|<COMMA>|<FULLSTOP>|<EXCLAMATION>|<QUESTION>|<SEMICOLON>|<COLON>|<OPENPAREN>|<CLOSEPAREN>|<OPENBRACKET>|<CLOSEBRACKET>|<OPENBRACE>|<CLOSEBRACE>|<QUOTE>|<DOUBLEQUOTE>|<DASH>|<UNDERLINE>|<ELLIPSIS>', text)

# Preprocess and tokenize the 'title' column
df['title'] = df['title'].apply(preprocess_text)
df['tokens'] = df['title'].apply(tokenize_text)

# Create a vocabulary (a dictionary mapping tokens to unique IDs)
def create_vocabulary(tokens_series):
    # Flatten the list of token lists into a single list
    all_tokens = [token for tokens in tokens_series for token in tokens]
    # Create a unique token list and assign an ID to each
    vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
    return vocabulary

# Get vocabulary from the tokens in the DataFrame
vocab = create_vocabulary(df['tokens'])

# Function to convert tokens to token IDs using the vocabulary
def tokens_to_ids(tokens, vocab):
    return [vocab[token] for token in tokens]

# Apply the function to convert tokens to token IDs
df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# Display the DataFrame with token IDs
print(df)


           id dead   type              by                 time  \
0     3150000  NaN  story     ColinWright  2011-10-24 16:27:00   
1     3500001  NaN  story            hncj  2012-01-23 11:39:25   
2     3150001  NaN  story      andymboyle  2011-10-24 16:27:36   
3     6050000  NaN  story         digisth  2013-07-16 05:16:26   
4      150000    t  story         jazzdev  2008-03-30 09:46:25   
...       ...  ...    ...             ...                  ...   
2638  4250098  NaN  story        aluciani  2012-07-16 11:46:03   
2639  4250101    t  story  davidhodges123  2012-07-16 11:46:34   
2640  4250105    t  story   SlipperySlope  2012-07-16 11:49:03   
2641  4250107    t  story    homewindow12  2012-07-16 11:49:38   
2642  4250109  NaN  story   SlipperySlope  2012-07-16 11:50:02   

                                                   text  parent  \
0                                                   NaN     NaN   
1                                                   NaN     NaN   
2     

In [149]:

#import pandas as pd
#import sentencepiece as spm


# Step 1: Preprocess the text (optional)
# def preprocess_text(text):
#     # Replace specific punctuation with special tokens
#     text = text.replace(',', '<COMMA>')  # Replace commas
#     text = text.replace('.', '<FULLSTOP>')  # Replace full stops
#     text = text.replace('!', '<EXCLAMATION>')  # Replace exclamations
#     return text

# # Preprocess the 'title' column
# df['title'] = df['title'].apply(preprocess_text)

# # Step 2: Save titles to a file for training SentencePiece
# with open('titles.txt', 'w') as f:
#     for title in df['title']:
#         f.write(title + '\n')

# # Step 3: Train a SentencePiece model
# # Increase vocab_size to at least 100 to avoid the error
# spm.SentencePieceTrainer.train('--input=titles.txt --model_prefix=m --vocab_size=100 --character_coverage=1.0')

# # Step 4: Load the trained SentencePiece model
# sp = spm.SentencePieceProcessor(model_file='m.model')

# # Tokenize the 'title' column using SentencePiece
# df['tokens'] = df['title'].apply(lambda x: sp.encode(x, out_type=str))

# # Create a vocabulary from the tokens
# def create_vocabulary(tokens_series):
#     all_tokens = [token for tokens in tokens_series for token in tokens]
#     vocabulary = {token: idx + 1 for idx, token in enumerate(sorted(set(all_tokens)))}
#     return vocabulary

# # Get vocabulary from the tokens in the DataFrame
# vocab = create_vocabulary(df['tokens'])

# # Function to convert tokens to token IDs using the vocabulary
# def tokens_to_ids(tokens, vocab):
#     return [vocab[token] for token in tokens]

# # Apply the function to convert tokens to token IDs
# df['token_ids'] = df['tokens'].apply(lambda tokens: tokens_to_ids(tokens, vocab))

# # Display the DataFrame with tokens and token IDs
# print(df)


# embedding 

We now need to get the embedding first we will analyze the embedding process for the wikipedia text before 

In [150]:
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download the required NLTK resources if you haven't already
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Replace specific punctuation with special tokens
    replacements = {
        ',': '<COMMA>',
        '.': '<FULLSTOP>',
        '!': '<EXCLAMATION>',
        '?': '<QUESTION>',
        ';': '<SEMICOLON>',
        ':': '<COLON>',
        '(': '<OPENPAREN>',
        ')': '<CLOSEPAREN>',
        '[': '<OPENBRACKET>',
        ']': '<CLOSEBRACKET>',
        '{': '<OPENBRACE>',
        '}': '<CLOSEBRACE>',
        '\'': '<QUOTE>',
        '\"': '<DOUBLEQUOTE>',
        '-': '<DASH>',
        '_': '<UNDERLINE>',
        '…': '<ELLIPSIS>'  # Ellipsis
    }

    # Replace punctuation with special tokens
    for punctuation, token in replacements.items():
        text = text.replace(punctuation, token)

    # Lowercase the text
    text = text.lower()
    
    return text

# Function to tokenize and apply stemming/lemmatization
#def tokenize_and_process(text):
    # Tokenize the text by whitespace
    tokens = text.split()
    
    # Apply stemming or lemmatization
    processed_tokens = [lemmatizer.lemmatize(stemmer.stem(token)) for token in tokens]
    
    return processed_tokens

# Function to tokenize without stemming/lemmatization
def tokenize_and_process(text):
    # Tokenize the text by whitespace
    tokens = text.split()
    
    return tokens


# Read the text file
file_path = 'text8.txt'  # Replace with your file path
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Preprocess the content
preprocessed_content = preprocess_text(content)

# Tokenize and process the preprocessed text
tokens = tokenize_and_process(preprocessed_content)

# Create sentences for Word2Vec
sentences = [tokens]  # Wrap tokens in a list to form a single sentence

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=0)

# Save the model
model.save("word2vec.model")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\omare\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [165]:
len(sentences)

1

In [152]:
# Example usage: getting the vector for a word
word_vector = model.wv['exampl']  # Replace 'example' with any word from your corpus

print(word_vector)

[ 2.2169328e-03  8.1457589e-03  2.5015855e-03  4.4307020e-03
 -4.2153504e-03 -8.9858631e-03 -1.5310884e-03  1.3215863e-03
 -8.2708237e-04 -3.5400926e-03 -6.4976024e-03  2.7700793e-03
 -6.0778796e-03  6.4819301e-03 -6.1873733e-03  6.1717201e-03
  4.2399121e-03 -8.8933287e-03  3.0457056e-03 -6.9996440e-03
  2.3178649e-03 -5.5011176e-03 -3.6389255e-03  1.3364435e-03
  6.2675681e-03  1.7851364e-03  2.8404749e-03 -4.4528805e-03
 -4.4578002e-03  8.7969489e-03  5.5197608e-03 -3.6655618e-03
  2.6506258e-03 -4.0250420e-04  1.9467354e-04  9.6652545e-03
  5.5776238e-03 -1.6557014e-03  4.7752871e-03 -5.9111891e-03
  8.1074676e-03 -2.9322363e-03 -5.8023944e-03 -3.9075706e-03
  5.9711919e-03  9.1314232e-03 -3.7671484e-03  3.9444994e-03
  8.0300914e-03 -6.0865879e-03  2.4882329e-03 -7.5024664e-03
 -6.8915915e-03 -5.3569768e-03  6.8802941e-03  9.0985297e-05
  9.9987574e-03 -9.4637321e-03  3.0739342e-03 -4.0175333e-03
 -9.6577499e-03 -3.0278838e-03 -2.1026398e-03 -4.0729572e-03
  4.1171825e-03  5.97615

In [153]:
import numpy as np
from scipy.stats import spearmanr
from gensim.models import Word2Vec

# Load your pre-trained Word2Vec model or train one
# model = Word2Vec.load("your_model_path")  # Load an existing model
# Or train your model
# model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Example word pairs and their human-annotated similarity scores
word_pairs = [('king', 'queen'), ('man', 'woman')]  # Example pairs
human_scores = [0.8, 0.9]  # Human-annotated scores

# Compute cosine similarities
similarities = []
for w1, w2 in word_pairs:
    try:
        vec1 = model.wv[w1]  # Get the vector for the first word
        vec2 = model.wv[w2]  # Get the vector for the second word
        cosine_similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
        similarities.append(cosine_similarity)
    except KeyError as e:
        print(f"Word not in vocabulary: {e}")

# Calculate Spearman correlation
if similarities:  # Ensure there are similarities to compare
    corr, _ = spearmanr(human_scores, similarities)
    print(f'Spearman correlation: {corr}')
else:
    print("No valid similarities calculated.")


Spearman correlation: -0.9999999999999999


In [154]:
# Example: Get word vectors for all tokens in the first row
first_row_vectors = [model.wv[token] for token in df['tokens'][10] if token in model.wv]

# Display word vectors
print("Word Vectors for the first row tokens:")
for token, vector in zip(df['tokens'][500], first_row_vectors):
    print(f"Token: {token}, Vector: {vector}")

Word Vectors for the first row tokens:
Token: the, Vector: [ 0.00215919 -0.00915128  0.00352259 -0.00158813 -0.00605567 -0.00716093
 -0.00900247  0.00796636 -0.00185247  0.00825693 -0.00636896 -0.00516181
 -0.00809662  0.00962218  0.00150697 -0.00340323 -0.00037912 -0.00676415
  0.00090843  0.00869125 -0.00113445 -0.0036642   0.00731904  0.00285193
 -0.00022883  0.00754815 -0.00137701 -0.00993149 -0.0034574   0.00358534
 -0.00541263 -0.00361407 -0.00507     0.00108166  0.00853553 -0.00947306
  0.00770152  0.0029809   0.00277802 -0.00991549  0.00115307 -0.00178094
 -0.00226703  0.00141892 -0.00702713 -0.0025684  -0.00595684  0.00045144
  0.00494801 -0.00108522  0.00096037 -0.00095726 -0.00519448 -0.00044592
 -0.0070453   0.00470832 -0.00162558 -0.00965235  0.00043421  0.00440209
  0.00909637 -0.00408069  0.00030179  0.00495305 -0.00971529  0.00876166
  0.00380845 -0.00994397  0.00737749 -0.00076256 -0.00283092 -0.00537754
  0.00346894 -0.00313984 -0.00986447 -0.00942327 -0.00095738  0.0

In [156]:
import numpy as np
from gensim.models import Word2Vec

# Load your trained Word2Vec model
# model = Word2Vec.load("your_model_path")  # Uncomment to load an existing model

# Function to predict a target word using CBOW
def predict_target_word(context_words, model):
    # Filter out words not in the model's vocabulary
    context_vectors = []
    for word in context_words:
        if word in model.wv:
            context_vectors.append(model.wv[word])
        else:
            print(f"Word '{word}' not in vocabulary.")

    # Check if we have valid context vectors
    if not context_vectors:
        return None
    
    # Average the context word vectors
    context_vector = np.mean(context_vectors, axis=0)

    # Find the most similar word to the context vector
    similar_words = model.wv.similar_by_vector(context_vector, topn=5)  # Get top 5 similar words

    return similar_words

# Example sentence and context related to chloroplasts
target_phrase = "chloroplasts are vital for photosynthesis"
context_words = ["chloroplasts", "are", "vital", "for", "photosynthesis"]

# Predicting the target word
predicted_words = predict_target_word(context_words, model)

# Display predicted words
if predicted_words:
    print("Predicted words for context:", predicted_words)
else:
    print("No valid predictions could be made.")


Predicted words for context: [('for', 0.9838205575942993), ('autism', 0.9808556437492371), ('autistic', 0.9801835417747498), ('are', 0.9790120720863342), ('or', 0.9789362549781799)]


# pytorch  


In [167]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Create a mapping from tokens to their indices
token_to_index = {token: idx for idx, token in enumerate(set(tokens))}
index_to_token = {idx: token for token, idx in token_to_index.items()}

# Convert tokens to their corresponding indices
numerical_data = [token_to_index[token] for token in tokens]

# Prepare training samples (you can use a sliding window approach or any method suitable for your task)
window_size = 2  # For context size
X = []
y = []

for i in range(window_size, len(numerical_data) - window_size):
    context = numerical_data[i - window_size:i] + numerical_data[i + 1:i + window_size + 1]
    target = numerical_data[i]
    X.append(context)
    y.append(target)

X = np.array(X)
y = np.array(y)


In [168]:
class Word2VecDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

# Create dataset and dataloader
dataset = Word2VecDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [169]:
import torch.nn as nn

class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear = nn.Linear(embedding_dim, vocab_size)

    def forward(self, context):
        # Get embeddings for context words
        embeds = self.embeddings(context)
        # Average the embeddings
        avg_embed = embeds.mean(dim=1)
        return self.linear(avg_embed)

# Initialize the model
vocab_size = len(token_to_index)
embedding_dim = 100  # Same as Word2Vec vector size
model = Word2VecModel(vocab_size, embedding_dim)


In [171]:
from tqdm import tqdm  # Import tqdm

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Wrap your DataLoader with tqdm to show progress
    with tqdm(total=len(dataloader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for context, target in dataloader:
            optimizer.zero_grad()  # Clear gradients
            output = model(context)  # Forward pass
            loss = criterion(output, target)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
            
            pbar.update(1)  # Update the progress bar
            pbar.set_postfix(loss=loss.item())  # Optionally display loss value

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')



Epoch 1/10:   0%|          | 1120/531413 [02:23<18:50:14,  7.82batch/s, loss=12]  


KeyboardInterrupt: 